In [9]:
# !pip install moexalgo

In [1]:
from moexalgo import Market, Ticker, session, CandlePeriod
from datetime import datetime, timedelta
import pandas as pd
import os
from dotenv import load_dotenv
import time
from colorama import Fore, Style
import requests


load_dotenv()


class Config:
    Login = os.getenv('LOGIN')  # Адрес электронной почты, указанный при регистрации на сайте moex.com
    Password = os.getenv('PASSWORD')  # Пароль от учетной записи на сайте moex.com

In [12]:
datetime.now().isoformat()

'2024-05-24T21:14:11.538673'

In [13]:
session.authorize(Config.Login, Config.Password)
session.AUTH_CERT

'OGrIvoojxvupKfqnfAq3LgUAAABRiA-wgGXseeYAhdLqXDfchSE2cZRFDVzUQfI2C6hffdQersTthTrvlmkk2nFTLPNQK_jazoF0ckNv3m0gz_eEz3iW4VnRnd4Hoyv1Dxj_-qpUrvhRgYbE30qS_-hKjDgKa0pBKIY5uvHeIN1m-Aq1CVcX-p4pC8fzAXrj6sMxhqAq8zogfZk1'

# Акции

In [14]:
# Акции SBER
sber = Ticker('SBER')
period = 5 # Период в минутах 1, 10, 60 или '1m', '10m', '1h', 'D', 'W', 'M', 'Q'; по умолчанию 60
start_date = '2023-10-10'
end_date = '2023-10-18'
# Все акции
stocks = Market('stocks')

In [15]:
def fetch_orderbook_data():
    os.system('cls' if os.name == 'nt' else 'clear')

    orderbook = sber.orderbook()
   
    df = pd.DataFrame(orderbook)
    columns_to_keep = ['secid', 'buysell', 'price', 'quantity', 'updatetime']
    filt_df = df[columns_to_keep]
   
    for index, row in filt_df.iterrows():
        if index == 9:
            print(f"{row['secid']} {row['buysell']} {Fore.GREEN}{row['price']}{Style.RESET_ALL} {row['quantity']} {row['updatetime']}")
        elif index == 10:
            print(f"{row['secid']} {row['buysell']} {Fore.RED}{row['price']}{Style.RESET_ALL} {row['quantity']} {row['updatetime']}")
        else:
            print(f"{row['secid']} {row['buysell']} {row['price']} {row['quantity']} {row['updatetime']}")



In [67]:
def fetch_trades_data(tradeno='10447501224', date=datetime.now().date().strftime('%Y-%m-%d')):
    url = f"https://iss.moex.com/iss/engines/stock/markets/shares/boards/tqbr/securities/sber/trades.json?tradeno={tradeno}&date={date}"

    # Define the cookies for authorization
    cookies = {
        'MicexPassportCert': session.AUTH_CERT
    }

    try:
        response = requests.get(url, cookies=cookies)
        response.raise_for_status()  # Raise an error for bad status codes
        # Print the JSON data
        data = response.json()
        
        return data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")

def get_last_trades_data():
    global last_tradeno;
    
    tradestats_data = fetch_trades_data(tradeno=last_tradeno)
    columns=tradestats_data['trades']['columns']
    # tradestats_data_pd = pd.DataFrame(data=tradestats_data['trades']['data'], columns=tradestats_data['trades']['columns'])
    # tradestats_data_pd.to_json('./tick_data1.json')
    
    trades_data = tradestats_data['trades']['data']
    last_tradeno = trades_data[-1][0]

    while len(trades_data) == 5000:
        trades_data = fetch_trades_data(tradeno=last_tradeno)['trades']['data']
        last_tradeno = trades_data[-1][0]


    return pd.DataFrame(data=trades_data, columns=columns)

def get_last_N_sec(df, period=5):
    df['TRADETIME'] = pd.to_timedelta(df['TRADETIME'])

    # Get the current time and convert to timedelta since start of the day
    now = datetime.now().time()
    current_time = pd.to_timedelta(now.strftime('%H:%M:%S'))
    
    # Calculate the threshold for the last 5 seconds
    threshold = current_time - pd.Timedelta(seconds=period)
    
    # Filter rows within the last 5 seconds
    last_5_seconds_df = df[df['TRADETIME'] > threshold]
    last_5_seconds_df.reset_index(inplace=True)
    
    return last_5_seconds_df

def price_shift(df):
    return (df['PRICE'][0] - df['PRICE'].iloc[-1])/df['PRICE'].iloc[-1]*100

def buy_sell(df):
    sums = df.groupby('BUYSELL')['QUANTITY'].sum()
    
    # Get the sum for 'B' and 'S', defaulting to 0 if they don't exist
    buy_sum = sums.get('B', 0)
    sell_sum = sums.get('S', 0)

    return buy_sum, sell_sum

In [17]:
last_tradeno = '10447501224'
trades_data = get_last_trades_data()
# print(trades_data)
get_last_N_sec(trades_data)

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION
953,10468165814,0 days 21:14:34,TQBR,SBER,321.5,1,3215.0,N,2114,2024-05-24 21:14:35,B,2,2


In [70]:
while True:
    # clear_output(wait=True)
    # print(sber.orderbook())
    # sber.orderbook()
    # print(fetch_trades_data("10447501224"))
    trades_data = get_last_trades_data()
    last_N_sec = get_last_N_sec(trades_data)
    print(last_N_sec)
    if len(last_N_sec) != 0:
        print(f'Value: {last_N_sec["VALUE"].sum()}; Volume: {last_N_sec["QUANTITY"].sum()};')
        print(f'Изменение цены: {price_shift(last_N_sec):.4f}')
    
        buy, sell = buy_sell(last_N_sec)
        print(f'Покупка: {buy/(buy+sell)*100:.2f}%, Продажа: {sell/(buy+sell)*100:.2f}%')
    print()

# display(sber.orderbook())
    time.sleep(5)
# print(sber.orderbook())


   index      TRADENO       TRADETIME BOARDID SECID   PRICE  QUANTITY   VALUE  \
0      5  10468229919 0 days 22:20:16    TQBR  SBER  321.43         1  3214.3   

  PERIOD  TRADETIME_GRP              SYSTIME BUYSELL  DECIMALS TRADINGSESSION  
0      N           2220  2024-05-24 22:20:17       S         2              2  
Value: 3214.3; Volume: 1;
Изменение цены: 0.0000
Покупка: 0.00%, Продажа: 100.00%

   index      TRADENO       TRADETIME BOARDID SECID   PRICE  QUANTITY   VALUE  \
0      1  10468230066 0 days 22:20:24    TQBR  SBER  321.43         1  3214.3   
1      2  10468230067 0 days 22:20:24    TQBR  SBER  321.38         1  3213.8   
2      3  10468230068 0 days 22:20:24    TQBR  SBER  321.36         3  9640.8   

  PERIOD  TRADETIME_GRP              SYSTIME BUYSELL  DECIMALS TRADINGSESSION  
0      N           2220  2024-05-24 22:20:25       S         2              2  
1      N           2220  2024-05-24 22:20:25       S         2              2  
2      N           2220  2024

KeyboardInterrupt: 

In [7]:
# add combined trade_date_time column to df
def trade_date_time(df):
    # string from tradedate
    df['tradedate_dt'] = df['tradedate'].apply(lambda item: item.strftime('%Y-%m-%d'))

    #string from tradetime
    df['tradetime_dt'] = pd.to_datetime(df['tradetime'].apply(lambda x: pd.Timestamp.combine(pd.Timestamp.today(), x)))
    df['tradetime_str'] = df['tradetime_dt'].dt.strftime('%H:%M:%S')
    df['trade_date_time'] = pd.to_datetime(df['tradedate_dt'] + ' ' + df['tradetime_str'])

    df.drop(['tradetime_dt', 'tradetime_str', 'tradedate_dt'], axis=1, inplace=True)

## Candles
<p>Классические свечи OHLCV. Гранулярность данных 1 мин (1m), 10 мин (10m), 1 час (60m), 1 день (1d)</p>

In [68]:
# Свечи по акциям SBER за период
sber.candles(start=start_date, end=end_date, period=CandlePeriod.TEN_MINUTES).head(5)
# sber.candles(date='2023-01-10', till_date=datetime.now().date(), period='1D').head(200)

,open,close,high,low,value,volume,begin,end
0,264.89,264.89,264.89,264.89,68129708.0,257200.0,2023-10-10 09:50:00,2023-10-10 09:59:59
1,264.90,264.25,265.18,263.80,573080891.3,2167670.0,2023-10-10 10:00:00,2023-10-10 10:09:59
2,264.26,264.22,264.95,263.90,441929827.4,1671130.0,2023-10-10 10:10:00,2023-10-10 10:19:59
3,264.22,264.23,264.45,264.05,187603360.0,710040.0,2023-10-10 10:20:00,2023-10-10 10:29:59
4,264.23,264.23,264.58,264.20,217393400.8,822500.0,2023-10-10 10:30:00,2023-10-10 10:39:59


## Tradestats
<p>Торговая статистика на основе <b>сделок</b>. Гранулярность данных 5 мин.</p>

In [97]:
# Данные по акциям SBER за период
sber.tradestats(start=start_date, end=end_date).head(5)

,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,...,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime
0,SBER,2023-10-10,10:05:00,264.90,265.18,264.43,264.46,0.0001,89034,235729237.0,...,1166,1200,108558101.0,127171135.0,40995,48039,-0.08,264.81,264.72,2023-10-26 15:44:37
1,SBER,2023-10-10,10:10:00,264.46,264.50,263.80,264.25,0.0001,127733,337351653.0,...,920,1388,112969121.0,224382532.0,42771,84962,-0.33,264.13,264.10,2023-10-26 15:44:38
2,SBER,2023-10-10,10:15:00,264.26,264.95,264.12,264.94,0.0001,88051,232913970.0,...,1059,567,188526086.0,44387884.0,71267,16784,0.62,264.53,264.47,2023-10-26 15:44:38
3,SBER,2023-10-10,10:20:00,264.91,264.94,263.90,264.22,0.0001,79062,209015857.0,...,610,1085,48952344.0,160063513.0,18509,60553,-0.53,264.48,264.34,2023-10-26 15:44:39
4,SBER,2023-10-10,10:25:00,264.22,264.35,264.05,264.10,0.0000,31648,83606467.0,...,574,549,44623014.0,38983453.0,16890,14758,0.07,264.20,264.15,2023-10-26 15:44:39


In [98]:
sber.tradestats(start=datetime.now().date(), end=datetime.now().date()).head(5)


,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,...,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime
0,SBER,2024-05-23,10:05:00,322.80,322.80,321.23,322.30,0.0001,84058,270599536.0,...,785,1005,126221919.0,144377618.0,39218,44840,-0.07,321.85,321.98,2024-05-23 10:05:12
1,SBER,2024-05-23,10:10:00,322.30,322.30,321.50,321.65,0.0000,30832,99230832.0,...,312,567,26796402.0,72434430.0,8323,22509,-0.46,321.96,321.80,2024-05-23 10:10:12
2,SBER,2024-05-23,10:15:00,321.64,321.81,321.52,321.80,0.0000,26100,83947330.0,...,318,228,49614273.0,34333057.0,15425,10675,0.18,321.65,321.62,2024-05-23 10:15:11
3,SBER,2024-05-23,10:20:00,321.81,322.32,321.77,322.25,0.0000,19548,62949272.0,...,387,386,28902259.0,34047012.0,8974,10574,-0.08,322.07,321.99,2024-05-23 10:20:11
4,SBER,2024-05-23,10:25:00,322.25,322.26,322.10,322.14,0.0000,7723,24880664.0,...,166,254,13821028.0,11059637.0,4290,3433,0.11,322.17,322.16,2024-05-23 10:25:11


In [26]:
# Данные по всем акциям за дату. Чтобы получить данные за период, нужно запустить цикл по датам
stocks.tradestats(date=start_date).head(10)

,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,...,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime
0,ABIO,2023-10-10,10:05:00,119.520,119.800,114.000,117.380,0.0017,45904,53480659.0,...,647,790,26741563.0,26739095.0,22932,22972,0.00,116.610,116.400,2023-10-26 15:44:37
1,ABRD,2023-10-10,10:05:00,305.600,307.400,305.600,306.600,0.0007,480,1471600.0,...,36,13,1141020.0,330580.0,372,108,0.55,306.700,306.100,2023-10-26 15:44:37
2,AFKS,2023-10-10,10:05:00,17.601,17.606,17.501,17.511,0.0003,10943,19184629.0,...,206,152,5859774.0,13324855.0,3341,7602,-0.39,17.539,17.528,2023-10-26 15:44:37
3,AFLT,2023-10-10,10:05:00,40.560,40.590,40.470,40.550,0.0004,4517,1830400.0,...,139,46,1352553.0,477847.0,3337,1180,0.48,40.530,40.500,2023-10-26 15:44:37
4,AGRO,2023-10-10,10:05:00,1200.000,1209.600,1200.000,1209.600,0.0004,5834,7036248.0,...,265,60,5784024.0,1252224.0,4795,1039,0.64,1206.300,1205.200,2023-10-26 15:44:37
5,AKRN,2023-10-10,10:05:00,18942.000,18986.000,18942.000,18950.000,0.0012,62,1174990.0,...,3,7,682228.0,492762.0,36,26,0.16,18951.000,18952.000,2023-10-26 15:44:37
6,ALRS,2023-10-10,10:05:00,75.010,75.080,74.910,74.940,0.0002,15084,11309084.0,...,341,271,5071387.0,6237697.0,6763,8321,-0.10,74.990,74.960,2023-10-26 15:44:37
7,AMEZ,2023-10-10,10:05:00,97.390,97.540,97.140,97.180,0.0008,39,379624.0,...,9,12,165578.0,214046.0,17,22,-0.13,97.399,97.294,2023-10-26 15:44:37
8,APTK,2023-10-10,10:05:00,16.552,16.616,16.288,16.408,0.0006,40882,6698078.0,...,262,414,2648451.0,4049626.0,16185,24697,-0.21,16.364,16.397,2023-10-26 15:44:37
9,AQUA,2023-10-10,10:05:00,954.000,956.000,954.000,955.500,0.0004,858,819663.0,...,32,21,675482.0,144181.0,707,151,0.65,955.400,954.800,2023-10-26 15:44:37


## Orderstats
<p>Торговая статистика на основе <b>заявок</b>. Гранулярность данных 5 мин.</p>

In [108]:
# Данные по акциям SBER за период
sber.orderstats(date=start_date, till_date='2023-10-18').head()

,ticker,tradedate,tradetime,put_orders_b,put_orders_s,put_val_b,put_val_s,put_vol_b,put_vol_s,put_vwap_b,put_vwap_s,put_vol,put_val,put_orders,cancel_orders_b,cancel_orders_s,cancel_val_b,cancel_val_s,cancel_vol_b,cancel_vol_s,cancel_vwap_b,cancel_vwap_s,cancel_vol,cancel_val,cancel_orders,systime
0,SBER,2023-10-10,10:05:00,12588,15586,4.008831e+09,5.575253e+09,1511580,2038427,265.21,273.51,3550007,9.584084e+09,28174,10700,14203,3.640648e+09,4.970981e+09,1375738,1862514,264.63,266.90,3238252,8.611629e+09,24903,2023-10-25 21:26:47
1,SBER,2023-10-10,10:10:00,5567,6791,1.940628e+09,2.657036e+09,714584,1001512,271.57,265.30,1716096,4.597664e+09,12358,3762,5593,1.463819e+09,2.290195e+09,545309,863399,268.44,265.25,1408708,3.754014e+09,9355,2023-10-25 21:26:47
2,SBER,2023-10-10,10:15:00,6289,5895,2.387144e+09,2.480124e+09,898471,933868,265.69,265.58,1832339,4.867268e+09,12184,5168,4910,2.092585e+09,2.292207e+09,781963,864446,267.61,265.16,1646409,4.384792e+09,10078,2023-10-25 21:26:48
3,SBER,2023-10-10,10:20:00,5045,5893,1.633560e+09,2.521517e+09,615955,953957,265.21,264.32,1569912,4.155077e+09,10938,4091,5134,1.478751e+09,2.284129e+09,559684,862257,264.21,264.90,1421941,3.762880e+09,9225,2023-10-25 21:26:50
4,SBER,2023-10-10,10:25:00,4320,4016,2.076028e+09,2.242749e+09,785969,846117,264.14,265.06,1632086,4.318777e+09,8336,3443,3336,1.883450e+09,2.157313e+09,713452,815408,263.99,264.57,1528860,4.040763e+09,6779,2023-10-25 21:26:51


In [106]:
# Данные по всем акциям за дату. Чтобы получить данные за период, нужно запустить цикл по датам
stocks.orderstats(date=start_date).head()

,ticker,tradedate,tradetime,put_orders_b,put_orders_s,put_val_b,put_val_s,put_vol_b,put_vol_s,put_vwap_b,put_vwap_s,put_vol,put_val,put_orders,cancel_orders_b,cancel_orders_s,cancel_val_b,cancel_val_s,cancel_vol_b,cancel_vol_s,cancel_vwap_b,cancel_vwap_s,cancel_vol,cancel_val,cancel_orders,systime
0,ABIO,2023-10-10,10:05:00,2406,2077,1.559303e+08,1.243118e+08,132711,108227,117.500,114.860,240938,2.802420e+08,4483,1081,738,7.143639e+07,5.936611e+07,61143,51396,116.830,115.510,112539,1.308025e+08,1819,2023-10-25 21:26:47
1,ABRD,2023-10-10,10:05:00,240,420,1.161129e+07,3.472757e+07,3858,7354,301.000,472.200,11212,4.633886e+07,660,157,65,1.014435e+07,3.987252e+06,3353,1295,302.500,307.900,4648,1.413160e+07,222,2023-10-25 21:26:47
2,AFKS,2023-10-10,10:05:00,10587,11087,2.419249e+09,3.034073e+09,1381787,1718945,17.508,17.651,3100732,5.453323e+09,21674,10005,10355,2.388973e+09,2.972192e+09,1364280,1691773,17.511,17.569,3056053,5.361165e+09,20360,2023-10-25 21:26:47
3,AFLT,2023-10-10,10:05:00,3763,4335,4.947816e+08,9.923537e+08,1232143,2325439,40.160,42.670,3557582,1.487135e+09,8098,3258,3475,4.704643e+08,9.055934e+08,1167973,2222986,40.280,40.740,3390959,1.376058e+09,6733,2023-10-25 21:26:47
4,AGRO,2023-10-10,10:05:00,2602,1764,6.107427e+08,4.015822e+08,496530,327792,1230.000,1225.100,824322,1.012325e+09,4366,2291,1536,5.779238e+08,3.901706e+08,481335,323243,1200.700,1207.100,804578,9.680944e+08,3827,2023-10-25 21:26:47


## OrderBookstats
<p>Торговая статистика на основе <b>стакана заявок</b>. Гранулярность данных 5 мин.</p>

In [9]:
# Данные по акциям SBER за период
sber.obstats(date=start_date, till_date='2023-10-18').head(10)

,ticker,tradedate,tradetime,spread_bbo,spread_lv10,spread_1mio,levels_b,levels_s,vol_b,vol_s,...,val_s,imbalance_vol_bbo,imbalance_val_bbo,imbalance_vol,imbalance_val,vwap_b,vwap_s,vwap_b_1mio,vwap_s_1mio,systime
0,SBER,2023-10-10,10:05:00,1.0,8.7,2.2,1387,856,349423,456963,...,1241102353,-0.11,-0.11,-0.13,-0.16,256.25,271.59,264.80,264.86,2023-10-27 02:53:53
1,SBER,2023-10-10,10:10:00,0.8,8.1,1.6,1401,922,393925,509661,...,1383499548,-0.04,-0.04,-0.13,-0.16,256.41,271.46,264.16,264.21,2023-10-27 02:53:54
2,SBER,2023-10-10,10:15:00,0.7,8.0,1.7,1450,939,445063,509068,...,1381519454,-0.20,-0.20,-0.07,-0.09,257.09,271.38,264.45,264.49,2023-10-27 02:53:54
3,SBER,2023-10-10,10:20:00,0.7,7.9,1.5,1473,968,454221,516908,...,1402256629,0.38,0.38,-0.06,-0.09,257.10,271.28,264.51,264.55,2023-10-27 02:53:55
4,SBER,2023-10-10,10:25:00,0.6,8.0,1.5,1447,997,463721,525658,...,1424980746,0.25,0.25,-0.06,-0.09,256.98,271.08,264.15,264.19,2023-10-27 02:53:55
5,SBER,2023-10-10,10:30:00,0.5,7.8,1.4,1451,998,485084,533381,...,1445593921,0.06,0.06,-0.05,-0.07,257.21,271.02,264.22,264.25,2023-10-27 02:53:55
6,SBER,2023-10-10,10:35:00,0.5,7.6,1.3,1459,997,505804,535482,...,1450871514,-0.28,-0.28,-0.03,-0.05,257.46,270.95,264.31,264.34,2023-10-27 02:53:56
7,SBER,2023-10-10,10:40:00,0.4,7.4,0.7,1467,1004,510402,547460,...,1483067918,0.15,0.15,-0.04,-0.06,257.49,270.90,264.30,264.32,2023-10-27 02:53:57
8,SBER,2023-10-10,10:45:00,0.4,7.4,0.7,1430,1029,488927,555780,...,1505050433,0.05,0.05,-0.06,-0.09,257.11,270.80,263.89,263.91,2023-10-27 02:53:57
9,SBER,2023-10-10,10:50:00,0.5,7.8,1.3,1434,1036,494964,561824,...,1521064530,-0.30,-0.30,-0.06,-0.09,257.03,270.74,263.86,263.90,2023-10-27 02:53:57


In [69]:
# Данные по всем акциям за дату. Чтобы получить данные за период, нужно запустить цикл по датам
stocks.obstats(date=start_date).head()

,open,close,high,low,value,volume,begin,end,ticker,tradedate,tradetime,spread_bbo,spread_lv10,spread_1mio,levels_b,levels_s,vol_b,vol_s,val_b,val_s,imbalance_vol_bbo,imbalance_val_bbo,imbalance_vol,imbalance_val,vwap_b,vwap_s,vwap_b_1mio,vwap_s_1mio,systime,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,trades,pr_vwap,pr_change,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime
0,264.89,264.89,264.89,264.89,6.812971e+07,257200.0,2023-10-10 09:00:00,2023-10-10 09:59:59,SBER,2023-10-10,10:05:00,1.0,8.7,2.2,1387,856,349423,456963,895411104,1241102353,-0.11,-0.11,-0.13,-0.16,256.25,271.59,264.80,264.86,2023-10-27 02:53:53,SBER,2023-10-10,10:05:00,264.90,265.18,264.43,264.46,0.0001,89034.0,235729237.0,2366.0,264.76,-0.1661,1166.0,1200.0,108558101.0,127171135.0,40995.0,48039.0,-0.08,264.81,264.72,2023-10-26 15:44:37
1,264.90,263.82,265.18,263.58,1.878115e+09,7106910.0,2023-10-10 10:00:00,2023-10-10 10:59:59,SBER,2023-10-10,10:10:00,0.8,8.1,1.6,1401,922,393925,509661,1010091477,1383499548,-0.04,-0.04,-0.13,-0.16,256.41,271.46,264.16,264.21,2023-10-27 02:53:54,SBER,2023-10-10,10:10:00,264.46,264.50,263.80,264.25,0.0001,127733.0,337351653.0,2308.0,264.11,-0.0794,920.0,1388.0,112969121.0,224382532.0,42771.0,84962.0,-0.33,264.13,264.10,2023-10-26 15:44:38


In [20]:
candles = sber.candles(date=start_date, till_date=end_date)
obstats = sber.obstats(date=start_date, till_date=end_date)
trade_date_time(obstats)
tradestats = sber.tradestats(date=start_date, till_date=end_date)
trade_date_time(tradestats)
orderstats = sber.orderstats(date=start_date, till_date=end_date)
trade_date_time(orderstats)

# Объединение данных в один DataFrame
common_df = pd.concat([candles, obstats, tradestats,], axis=1)

common_df.head(2)

,open,close,high,low,value,volume,begin,end,ticker,tradedate,tradetime,spread_bbo,spread_lv10,spread_1mio,levels_b,levels_s,vol_b,vol_s,val_b,val_s,imbalance_vol_bbo,imbalance_val_bbo,imbalance_vol,imbalance_val,vwap_b,vwap_s,vwap_b_1mio,vwap_s_1mio,systime,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,trades,pr_vwap,pr_change,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime
0,264.89,264.89,264.89,264.89,6.812971e+07,257200.0,2023-10-10 09:00:00,2023-10-10 09:59:59,SBER,2023-10-10,10:05:00,1.0,8.7,2.2,1387,856,349423,456963,895411104,1241102353,-0.11,-0.11,-0.13,-0.16,256.25,271.59,264.80,264.86,2023-10-27 02:53:53,SBER,2023-10-10,10:05:00,264.90,265.18,264.43,264.46,0.0001,89034.0,235729237.0,2366.0,264.76,-0.1661,1166.0,1200.0,108558101.0,127171135.0,40995.0,48039.0,-0.08,264.81,264.72,2023-10-26 15:44:37
1,264.90,263.82,265.18,263.58,1.878115e+09,7106910.0,2023-10-10 10:00:00,2023-10-10 10:59:59,SBER,2023-10-10,10:10:00,0.8,8.1,1.6,1401,922,393925,509661,1010091477,1383499548,-0.04,-0.04,-0.13,-0.16,256.41,271.46,264.16,264.21,2023-10-27 02:53:54,SBER,2023-10-10,10:10:00,264.46,264.50,263.80,264.25,0.0001,127733.0,337351653.0,2308.0,264.11,-0.0794,920.0,1388.0,112969121.0,224382532.0,42771.0,84962.0,-0.33,264.13,264.10,2023-10-26 15:44:38
2,263.82,263.38,263.98,262.80,1.319014e+09,5009850.0,2023-10-10 11:00:00,2023-10-10 11:59:59,SBER,2023-10-10,10:15:00,0.7,8.0,1.7,1450,939,445063,509068,1144248313,1381519454,-0.20,-0.20,-0.07,-0.09,257.09,271.38,264.45,264.49,2023-10-27 02:53:54,SBER,2023-10-10,10:15:00,264.26,264.95,264.12,264.94,0.0001,88051.0,232913970.0,1626.0,264.52,0.2573,1059.0,567.0,188526086.0,44387884.0,71267.0,16784.0,0.62,264.53,264.47,2023-10-26 15:44:38
3,263.38,263.86,263.98,263.03,5.413183e+08,2054260.0,2023-10-10 12:00:00,2023-10-10 12:59:59,SBER,2023-10-10,10:20:00,0.7,7.9,1.5,1473,968,454221,516908,1167830852,1402256629,0.38,0.38,-0.06,-0.09,257.10,271.28,264.51,264.55,2023-10-27 02:53:55,SBER,2023-10-10,10:20:00,264.91,264.94,263.90,264.22,0.0001,79062.0,209015857.0,1695.0,264.37,-0.2605,610.0,1085.0,48952344.0,160063513.0,18509.0,60553.0,-0.53,264.48,264.34,2023-10-26 15:44:39
4,263.86,264.22,264.22,263.60,4.564655e+08,1729310.0,2023-10-10 13:00:00,2023-10-10 13:59:59,SBER,2023-10-10,10:25:00,0.6,8.0,1.5,1447,997,463721,525658,1191675990,1424980746,0.25,0.25,-0.06,-0.09,256.98,271.08,264.15,264.19,2023-10-27 02:53:55,SBER,2023-10-10,10:25:00,264.22,264.35,264.05,264.10,0.0000,31648.0,83606467.0,1123.0,264.18,-0.0454,574.0,549.0,44623014.0,38983453.0,16890.0,14758.0,0.07,264.20,264.15,2023-10-26 15:44:39


In [87]:
pd.set_option('display.max_columns', 52)
common_df.head(5)


In [88]:
trade_date_time(obstats)

In [90]:
obstats['trade_date_time'][0]

Timestamp('2023-10-10 10:05:00')

In [101]:
candles1 = sber.candles(date='2023-01-01', till_date='2024-01-01', period='1D')


In [110]:
extra = candles1[candles1['volume'] > 100000000]
print(len(extra))
extra.head(3)

18


,open,close,high,low,value,volume,begin,end
6,142.97,149.96,149.98,142.87,1.809715e+10,123147040.0,2023-01-11,2023-01-11 23:59:59
27,164.33,165.31,167.47,161.27,1.760888e+10,107139220.0,2023-02-09,2023-02-09 23:59:59
31,160.94,156.25,161.12,153.83,1.799286e+10,113804160.0,2023-02-15,2023-02-15 23:59:59


# Валюты

In [30]:
fx = Market('FX')
fx.tickers()

,ticker,shortname,lotsize,decimals,minstep
0,AEDRUBTODTOM,AED_TODTOM,100000,4,0.0001
1,AEDRUB_SPT,AEDRUB_SPT,1000,4,0.0025
2,AEDRUB_TMS,AEDRUB_TMS,1,4,0.0025
3,AEDRUB_TOD,AEDRUB_TOD,1000,4,0.0025
4,AEDRUB_TOM,AEDRUB_TOM,1000,4,0.0025
...,...,...,...,...,...
194,UZSRUB_TOM,UZSRUB_TOM,1000000,4,0.0025
195,ZARRUBTODTOM,ZAR_TODTOM,1000000,4,0.0001
196,ZARRUB_TMS,ZARRUB_TMS,1,4,0.0025
197,ZARRUB_TOD,ZARRUB_TOD,10000,4,0.0025


In [53]:
USDRUB_TMS = Ticker('USDRUB_TMS')
USDRUB_TMS.candles(start=datetime.now().date() - timedelta(days=3), end=datetime.now().date())

,open,close,high,low,value,volume,begin,end
0,91.7375,91.7575,91.7575,91.6900,114563.63,124900.0,2024-05-10 07:00:00,2024-05-10 07:09:59
1,91.7650,91.9725,91.9925,91.6900,16420.42,17865.0,2024-05-10 07:10:00,2024-05-10 07:19:59
2,91.7625,91.7375,91.7700,91.7375,10733.51,11700.0,2024-05-10 07:30:00,2024-05-10 07:39:59
3,91.7750,91.8125,91.8125,91.7750,41035.15,44700.0,2024-05-10 07:40:00,2024-05-10 07:49:59
4,91.9000,91.9000,91.9000,91.9000,1010.90,1100.0,2024-05-10 07:50:00,2024-05-10 07:57:30
...,...,...,...,...,...,...,...,...
66,92.2700,92.2650,92.3500,92.2625,65092.69,70541.0,2024-05-10 18:10:00,2024-05-10 18:19:59
67,92.2650,92.2000,92.3225,92.2000,38091.09,41282.0,2024-05-10 18:20:00,2024-05-10 18:29:59
68,92.1975,92.3100,92.3275,92.1825,124171.44,134500.0,2024-05-10 18:30:00,2024-05-10 18:39:59
69,92.1700,92.1875,92.3150,92.1700,49908.70,54084.0,2024-05-10 18:40:00,2024-05-10 18:49:59


In [211]:
dataa = {
    'time': ['12:00:01', '12:00:02', '12:00:03', '12:00:01', '12:00:02', '12:00:03', '12:00:01'],
    'value': [10, 20, 30, 40, 50, 60, 70]
}

dff = pd.DataFrame(dataa)

# Convert 'time' column to string if necessary (assuming it's in timedelta or datetime format)
# If the time is already in string format, you can skip this step
# df['time'] = df['time'].astype(str)

# Group by 'time' and sum the 'value' column
grouped_df = dff.groupby('time', as_index=False)['value'].sum()

print(grouped_df[grouped_df['value'] == 120]['time'][0])

12:00:01
